In [ ]:
import openai
openai.api_key = open("apikey.txt", "r").read().strip("\n")

## Experiment 1: Detecting opposing sentiments in scientific articles

`Description:` Can gpt3.5-turbo detect opposing sentiments in scientific papers? 

`Abilities Required:`
 - Parsing a potentially noisy paragraph sentence by sentence
 - Detecting opposing sentiments in scientific contexts
 - Identifying formal and informal references/mentions 

`Extra condition:` I wanted to make sure that the output were the same on two different occasions.

`Last updated`: 2023-04-11

In [3]:
message_history = []

def chat(inp, role="user"):
    message_history.append({"role": role, "content": inp})
    
    completion = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=message_history
    )

    reply_content = completion.choices[0].message.content
    print(reply_content)
    message_history.append({"role": "assistant", "content": reply_content})
    return reply_content

for i in range(4):
    user_input = input(">: ")
    print("User's input was", user_input)
    print()
    chat(user_input)
    print()

User's input was For the following sentence, provide a sentiment score from -1 to 1 (very negative, negative, slightly negative, slightly positive, positive, very positive). Limit your answer to the scores, without any explanation:\n\n"It maybe argued- and this is the line taken by W.-E.- both in the paper mentioned and in the book under review - that Lack's view rests on a mistaken notion of natural selection."

0.2 (slightly positive)

User's input was Can you do it again, but now for each sentence in this paragraph. Keep track of the sentences, the scores, and the sentiment label in a json:\n\n"It maybe argued- and this is the line taken by W.-E.- both in the paper mentioned and in the book under review - that Lack's view rests on a mistaken notion of natural selection.The process of selection works,not only between individuals,but also between groups and between species, thus promoting characters which are to the benefit of the group, even in the face of contrary individual selecti

In [8]:
message_history = []

prompts = [
    "For the following sentence, provide a sentiment score from -1 to 1 (very negative, negative, slightly negative, slightly positive, positive, very positive). Limit your answer to the scores, without any explanation:\n\n'It maybe argued- and this is the line taken by W.-E.- both in the paper mentioned and in the book under review - that Lack's view rests on a mistaken notion of natural selection.'",
    "Can you do it again, but now for each sentence in this paragraph. Keep track of the sentences, the scores, and the sentiment label in a json:\n\n'It maybe argued- and this is the line taken by W.-E.- both in the paper mentioned and in the book under review - that Lack's view rests on a mistaken notion of natural selection.The process of selection works,not only between individuals,but also between groups and between species, thus promoting characters which are to the benefit of the group, even in the face of contrary individual selection which may be kept in check by special devices. The existence of such evolutionary mechanisms is not only called for by a great many other wise unaccountable facts,but is also in accordance with genetic evolutionary theory (see e.g. WRIGHT, 1945)'",
    "If there are mentions or references to other authors in the sentence, provide sentiment scores for each of them too. Make sure to correctly identify when there are opposite sentiments within each sentence.",
    "Are you sure you are not missing references or mentions?"
]

for user_input in prompts:
    print("User's input was", user_input)
    print()
    chat(user_input)
    print()

User's input was For the following sentence, provide a sentiment score from -1 to 1 (very negative, negative, slightly negative, slightly positive, positive, very positive). Limit your answer to the scores, without any explanation:

'It maybe argued- and this is the line taken by W.-E.- both in the paper mentioned and in the book under review - that Lack's view rests on a mistaken notion of natural selection.'

0.0

User's input was Can you do it again, but now for each sentence in this paragraph. Keep track of the sentences, the scores, and the sentiment label in a json:

'It maybe argued- and this is the line taken by W.-E.- both in the paper mentioned and in the book under review - that Lack's view rests on a mistaken notion of natural selection.The process of selection works,not only between individuals,but also between groups and between species, thus promoting characters which are to the benefit of the group, even in the face of contrary individual selection which may be kept in 

In [ ]:
# validation
condition_1 = {
  "1": {
    "sentence": "It maybe argued- and this is the line taken by W.-E.- both in the paper mentioned and in the book under review - that Lack's view rests on a mistaken notion of natural selection.",
    "score": 0.2,
    "label": "slightly positive",
    "authors": {
      "W.-E.": {
        "score": 0.2,
        "label": "slightly positive"
      },
      "Lack": {
        "score": -0.3,
        "label": "slightly negative"
      }
    }
  },
  "2": {
    "sentence": "The process of selection works,not only between individuals,but also between groups and between species, thus promoting characters which are to the benefit of the group, even in the face of contrary individual selection which may be kept in check by special devices.",
    "score": 0.0,
    "label": "neutral",
    "authors": {}
  },
  "3": {
    "sentence": "The existence of such evolutionary mechanisms is not only called for by a great many other wise unaccountable facts,but is also in accordance with genetic evolutionary theory (see e.g. WRIGHT, 1945)",
    "score": 0.3,
    "label": "slightly positive",
    "authors": {
      "WRIGHT": {
        "score": 0.3,
        "label": "slightly positive"
      }
    }
  }
}

condition_2 = [
  {
    "sentence": "It maybe argued- and this is the line taken by W.-E.- both in the paper mentioned and in the book under review - that Lack's view rests on a mistaken notion of natural selection.",
    "score": 0.0,
    "label": "neutral",
    "authors": [
      {
        "name": "W.-E.",
        "score": 0.0,
        "label": "neutral"
      },
      {
        "name": "Lack",
        "score": -0.5574,
        "label": "negative"
      }
    ]
  },
  {
    "sentence": "The process of selection works,not only between individuals,but also between groups and between species, thus promoting characters which are to the benefit of the group, even in the face of contrary individual selection which may be kept in check by special devices.",
    "score": 0.4215,
    "label": "positive",
    "authors": []
  },
  {
    "sentence": "The existence of such evolutionary mechanisms is not only called for by a great many other wise unaccountable facts,but is also in accordance with genetic evolutionary theory (see e.g. WRIGHT, 1945)",
    "score": 0.4019,
    "label": "positive",
    "authors": [
      {
        "name": "WRIGHT",
        "score": 0.4019,
        "label": "positive"
      }
    ]
  }
]

The invididual promp v. the for loop approach gives slightly different results. Initerestingly, the difference in valence in the first sentence remains approximately the same ($0.2 - (-0.3) = 0.5 \approx 0 - (-0.5574) = 0.55$ ), but shifted on the negative side. That said, the output for the next two prompts become positive for no obvious reasons. I feel the first approach is closer to the right score than the second one in this case.

This is not catastrophic, but we can see how very similar approach can easily lead to change in output. 

`Ideas:`
 - Maybe we should anchored our prompt towards more neutral statement?

## Billing

User prompt condition?:
```
1:00 p.m.
Local time: Apr 11, 2023, 9:00 a.m.
gpt-3.5-turbo-0301, 4 requests
1,787 prompt + 590 completion = 2,377 tokens
```


For loop condition?:
```
3:50 p.m.
Local time: Apr 11, 2023, 11:50 a.m.
gpt-3.5-turbo-0301, 4 requests
1,732 prompt + 860 completion = 2,592 tokens
```

```
gpt-3.5-turbo	$0.002 / 1K tokens
```

In [10]:
def calc_price(nb_toks):
    return print(f"{round((nb_toks / 1000) * 0.002, 4)}$")

calc_price(2_592)

0.0052$
